# **Week 2(7/27-8/2)**  

Completed tasks: (9 hours)  
1 process data (1 hours)   
1.1 divided dataset into a balanced train(80%)/validation(10%)/test(10%) dataset based on "SA1.35" attribute. And saved splited result as "ids_split.pkl"   
1.2 generated brain connectivity maxtrix as the dataset of our project. And saved train/validation/test dataset as "con_mat_train.npy","con_mat_test.npy","con_mat_val.npy" 
1.3 generated brain connectivity maxtrix with all 853 subjects and saved it as "con_mat_all.npy" 

835 subects


2 Revise the code (8 hours)  
2.1 revised dataset class for brain connectivity matrix   
2.2 revised model constructure for our dataset  
2.3 revised loss function method  
2.4 revised training python file   
I borrowed codes from https://github.com/libilab/rsfMRI-VAE and https://github.com/rtqichen/beta-tcvae.  
To train a model:  
      python vae_quant.py --dataset brain --num_workers 4 --latent-dim 10 --batch-size 16 --num-epochs 150 --beta 15 --conv --tcvae --gpu 0 --save tcvae_10_15 --model_name tcvae_10_15  
   
We can modify beta and latent-dim values to search optimal parameters.

Questions and plans:  
1 how to upload codes to discovery conviniently after modifying? Now, every time, I delete a file on Discovery and then upload it.




# report details

# process data

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

In [ ]:
!python train.py ---

In [4]:
sub_attr = pd.read_spss(r"C:\Users\Administrator\Desktop\vae\data\AOMIC_Political.sav").astype('str')
roi = pd.read_pickle(r"C:\Users\Administrator\Desktop\vae\data\TimeData_aal_116.pkl")

In [5]:
ids_roi = set(roi.keys())       
ids_sub_attr = set(sub_attr['ID1000ID'].astype('int'))

In [6]:
ids_intersection = list(ids_roi.intersection(ids_sub_attr))

In [27]:
#save ids (totally 853)
with open("ids_inter.pkl", 'wb') as text:
    pickle.dump(ids_intersection, text)

In [71]:
#853 sub_attr_new
sub_attr['ID1000ID'] = sub_attr['ID1000ID'].astype('int')
sub_attr_new = sub_attr[sub_attr['ID1000ID'].isin(ids_intersection)]  
sub_attr_new = sub_attr_new.replace('nan','')

split data into train(80%), val(10), test(10)

In [53]:
def split_stratified_into_train_val_test(df_input, stratify_colname='y',
                                         frac_train=0.6, frac_val=0.15, frac_test=0.25,
                                         random_state=None):
    '''
    Splits a dataframe into three subsets (train, val, and test)
    following fractional ratios provided by the user, where each subset is
    stratified by the values in a specific column (that is, each subset has
    the same relative frequency of the values in the column).
    '''

    if frac_train + frac_val + frac_test != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % \
                         (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    # Split the temp dataframe into val and test dataframes.
    relative_frac_test = frac_test / (frac_val + frac_test)
    df_val, df_test, y_val, y_test = train_test_split(df_temp,
                                                      y_temp,
                                                      stratify=y_temp,
                                                      test_size=relative_frac_test,
                                                      random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val) + len(df_test)

    return df_train, df_val, df_test

In [77]:
df_train, df_val, df_test = split_stratified_into_train_val_test(sub_attr_new, stratify_colname='SA1.35', frac_train=0.80, frac_val=0.10, frac_test=0.10)

In [78]:
df_train['SA1.35'].value_counts()

           576
Links       57
 Rechts     49
Name: SA1.35, dtype: int64

In [79]:
df_val['SA1.35'].value_counts()

           72
Links       7
 Rechts     6
Name: SA1.35, dtype: int64

In [80]:
df_test['SA1.35'].value_counts()

           72
 Rechts     7
Links       7
Name: SA1.35, dtype: int64

In [86]:
ids_split = {
    'train' : list(df_train['ID1000ID']),
    'val' : list(df_val['ID1000ID']),
    'test' : list(df_test['ID1000ID']),
            }
with open("ids_split.pkl", 'wb') as text:
    pickle.dump(ids_split, text)

In [87]:
#with open("ids_split.pkl", 'rb') as text:
#    ids_split=pickle.load(text)

generate brain connectivity maxtrix

In [95]:
corr_list_train = []
for id in ids_split["train"]:
    #print(id)
    corr = np.corrcoef(roi[id].transpose())
    corr = np.expand_dims(corr, axis=0) 
    corr_list_train.append(corr)    
corr_all_train = np.concatenate(corr_list_train, axis=0) 

In [96]:
corr_list_val = []
for id in ids_split["val"]:
    #print(id)
    corr = np.corrcoef(roi[id].transpose())
    corr = np.expand_dims(corr, axis=0) 
    corr_list_val.append(corr)    
corr_all_val = np.concatenate(corr_list_val, axis=0) 

In [97]:
corr_list_test = []
for id in ids_split["test"]:
    #print(id)
    corr = np.corrcoef(roi[id].transpose())
    corr = np.expand_dims(corr, axis=0) 
    corr_list_test.append(corr)    
corr_all_test = np.concatenate(corr_list_test, axis=0) 

In [10]:
len(ids_intersection)
corr_list_all = []
for id in ids_intersection:
    #print(id)
    corr = np.corrcoef(roi[id].transpose())
    corr = np.expand_dims(corr, axis=0) 
    corr_list_all.append(corr)    
corr_all_all = np.concatenate(corr_list_all, axis=0) 

In [104]:
with open('con_mat_train.npy', 'wb') as f:
    np.save(f, corr_all_train)
with open('con_mat_val.npy', 'wb') as f:
    np.save(f, corr_all_val)
with open('con_mat_test.npy', 'wb') as f:
    np.save(f, corr_all_test)
with open('con_mat_all.npy', 'wb') as f:
    np.save(f, corr_all_all)

In [107]:
with open('con_mat_test.npy', 'rb') as f:
    a = np.load(f)
print(a.shape)

(86, 116, 116)


# constructure of the model


In [1]:
  (encoder): ConvEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1), padding=(6, 6))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv5): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_z): Conv2d(256, 20, kernel_size=(8, 8), stride=(1, 1))
    (act): ReLU(inplace=True)
  )
  (decoder): ConvDecoder(
    (conv1): ConvTranspose2d(10, 256, kernel_size=(8, 8), stride=(1, 1))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): ConvTranspose2d(256, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv4): ConvTranspose2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv5): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_final): ConvTranspose2d(64, 1, kernel_size=(1, 1), stride=(1, 1), padding=(6, 6))
    (act): ReLU(inplace=True)
  )

SyntaxError: invalid syntax (Temp/ipykernel_5292/2442007266.py, line 2)

The size of input and output is 1 * 116 * 116